In [1]:
import sympy
from sympy.parsing.sympy_parser import parse_expr
from ipywidgets import interact, FloatRangeSlider
import plotnine as p9
import pandas as pd
from math import pi

In [2]:
x = sympy.Symbol('x')

In [3]:
def factorial(n):
    """
        Calculates factorial
    """
    result = n
    if n == 1 or n == 0:
        return 1
    while n > 1:
        result *= (n-1)
        n -= 1
    return result

In [4]:
def n_derivative(expression, n: int):
    """
        Calculates n'th derivative
    """
    derivative = expression
    while n > 0:
        derivative = sympy.diff(derivative)
        n -= 1
    return derivative

In [5]:
def taylor_series(expression, n: int, around: float):
    """
        Calculates Taylor series
        n - order / number of terms
        around - approximation around certain point; real or complex number
    """
    derr_sum = 0
    while n >= 0:
        derr_sum += (
            n_derivative(expression, n).evalf(subs={'x': around})
            * ((x - around) ** n)
            / factorial(n)
        )
        n -= 1
    return derr_sum


taylor_series(sympy.cos(x), 4, 0)

0.0416666666666667*x**4 - 0.5*x**2 + 1.0

In [6]:
type(type(sympy.sin(x)))

sympy.core.function.FunctionClass

In [7]:
def create_dataset(expression, n, around):
    """
        
    """
    a = pd.Series(range(0, 1000+1, 1))/1000 * 4 * pi - 2 * pi
    expr = parse_expr(expression)
    def evaluate(x):
        return float(expr.evalf(subs={'x': x}))
    sin_data = pd.DataFrame(data={
        # create sample data
        'x': a,
        'f(x)': a.apply(evaluate),
        "f_approx": a.apply(
            sympy.lambdify(
                x,
                taylor_series(
                    expression=expr,
                    n=n,
                    around=around
                )
            )
        )
    })
    return sin_data

In [8]:
@interact(
    expression='sin(x)',
    n=(0, 10, 1),
    around=(-4, 4, 0.1),
    y_display_range=FloatRangeSlider(
        value=[-1.5, 1.5],
        min=-10.0,
        max=10.0,
        step=0.5,
        description='Y axis range'
    )
)
def create_plot(expression, n, around, y_display_range):
    data = create_dataset(expression, n, around)
    y_values = data[['f_approx', 'f(x)']]
    data = data[
        (y_values.min(axis=1) > y_display_range[0])
        &
        (y_values.max(axis=1) < y_display_range[1])
    ]
    plot = (
        p9.ggplot(data=data)
        + p9.aes(x="x")
        + p9.geom_point(color="red", mapping=p9.aes(y="f(x)"))
        + p9.geom_point(color="blue", alpha=0.15, mapping=p9.aes(y="f_approx"))
        + p9.ylim(y_display_range)
    )
    plot.draw()

interactive(children=(Text(value='sin(x)', description='expression'), IntSlider(value=5, description='n', max=…